# 言語処理100本ノック 2020年

[言語処理100本ノック2020版](https://nlp100.github.io/ja/)が公開されたのでこの機会に解いていく。
GitHubに公開しているJupyterをmarkdown出力したものなので，一度読み込んだパッケージはその後に読み込んでいないことに注意。

## 第3章：正規表現

> Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．
  1行に1記事の情報がJSON形式で格納される
  各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクト　に格納され，そのオブジェクトがJSON形式で書き出される
  ファイル全体はgzipで圧縮される
  以下の処理を行うプログラムを作成せよ．

内容的には2015と変わらないらしい。
Wikiのマークアップは[ここ](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)にまとめられている。

### 20. JSONデータの読み込み

> Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．


In [201]:
import json

def return_article(fname, article_title):
    with open(fname, 'rt') as data_file:
        for line in data_file:
            data_json = json.loads(line)
            if data_json['title'] == article_title:
                return data_json['text']

file_path = '../data/jawiki-country.json'
uk_article = return_article(file_path, 'イギリス')

print(uk_article[0:400])

{{redirect|UK}}
{{redirect|英国|春秋時代の諸侯国|英 (春秋)}}
{{Otheruses|ヨーロッパの国|長崎県・熊本県の郷土料理|いぎりす}}
{{基礎情報 国
|略名  =イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）


### 21. カテゴリ名を含む行を抽出

> 記事中でカテゴリ名を宣言している行を抽出せよ．

上記結果の中身を見るとカテゴリは以下のように記載されていた。

```
[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]
```

形式は`[[Category:カテゴリ名|ソートキー]]`という形をとっている。
特殊文字を，その特殊な意味を発動させずに使うには，バックスラッシュを使う必要がある。自分は`r'^\[+Category\:.+\]+$'`のように記載した。
`re.MULTILINE`とfindallを使うことで，改行ごとにforループを回さなくても検索することができる。

In [25]:
import re

def extract_category_row(wiki_text):
    p = re.compile(r'^\[+Category\:.+\]+$', re.MULTILINE)
    return p.findall(wiki_text)

category_rows = extract_category_row(uk_article)
for line in category_rows:
    print(line)

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


### 22. カテゴリ名の抽出

> 記事のカテゴリ名を(行単位ではなく名前で)抽出せよ。

`()`で囲んだ部分だけを抽出することができる。
`\w`でUnicodeの単語文字にマッチさせている。

In [32]:
def extract_category_name(wiki_text):
    p = re.compile(r'^\[+Category\:(\w+).+$', re.MULTILINE)
    return p.findall(wiki_text)

category_name = extract_category_name(uk_article)
for line in category_name:
    print(line)

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
現存する君主国
島国
1801年に成立した国家


### 23. セクション構造

> 記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ

==歴史==
こういうのがセクションらしい。

In [208]:
def extract_section(wiki_text):
    result = {}
    p = re.compile(r'^(={2,})(\w+)\1$', re.MULTILINE)
    section_content =  p.findall(wiki_text)
    for item in section_content:
        result[item[1]] = len(item[0])
    return result

section_dict = extract_section(uk_article)

for k,v in section_dict.items():
    print('レベル:',v, k)

レベル: 2 国名
レベル: 2 歴史
レベル: 2 地理
レベル: 3 主要都市
レベル: 3 気候
レベル: 2 政治
レベル: 3 元首
レベル: 3 法
レベル: 3 内政
レベル: 3 地方行政区分
レベル: 2 経済
レベル: 3 鉱業
レベル: 3 農業
レベル: 3 貿易
レベル: 3 不動産
レベル: 3 エネルギー政策
レベル: 3 通貨
レベル: 3 企業
レベル: 4 通信
レベル: 2 交通
レベル: 3 道路
レベル: 3 鉄道
レベル: 3 海運
レベル: 3 航空
レベル: 2 科学技術
レベル: 2 国民
レベル: 3 言語
レベル: 3 宗教
レベル: 3 婚姻
レベル: 3 移住
レベル: 3 教育
レベル: 3 医療
レベル: 2 文化
レベル: 3 食文化
レベル: 3 文学
レベル: 3 哲学
レベル: 3 音楽
レベル: 4 ポピュラー音楽
レベル: 3 映画
レベル: 3 コメディ
レベル: 3 国花
レベル: 3 世界遺産
レベル: 3 祝祭日
レベル: 3 スポーツ
レベル: 4 サッカー
レベル: 4 クリケット
レベル: 4 競馬
レベル: 4 モータースポーツ
レベル: 4 野球
レベル: 2 脚注
レベル: 2 関連項目
レベル: 2 外部リンク


### 24. ファイル参照の抽出

> 記事から参照されているメディアファイルをすべて抜き出せ.

`[[ファイル:Wikipedia-logo-v2-ja.png|thumb|説明文]]`というのがファイルの記載マークアップらしい。

In [86]:
def extract_file(wiki_text):
    p = re.compile(r'\[\[ファイル\:(.+?)\|')
    file_name = p.findall(wiki_text)
    return file_name

file_reference = extract_file(uk_article)
print(file_reference)

['Royal Coat of Arms of the United Kingdom.svg', 'Descriptio Prime Tabulae Europae.jpg', "Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg", 'London.bankofengland.arp.jpg', 'Battle of Waterloo 1815.PNG', 'Uk topo en.jpg', 'BenNevis2005.jpg', 'Population density UK 2011 census.png', '2019 Greenwich Peninsula & Canary Wharf.jpg', 'Birmingham Skyline from Edgbaston Cricket Ground crop.jpg', 'Leeds CBD at night.jpg', 'Glasgow and the Clyde from the air (geograph 4665720).jpg', 'Palace of Westminster, London - Feb 2007.jpg', 'Scotland Parliament Holyrood.jpg', 'Donald Trump and Theresa May (33998675310) (cropped).jpg', 'Soldiers Trooping the Colour, 16th June 2007.jpg', 'City of London skyline from London City Hall - Oct 2008.jpg', 'Oil platform in the North SeaPros.jpg', 'Eurostar at St Pancras Jan 2008.jpg', 'Heathrow Terminal 5C Iwelumo-1.jpg', 'Airbus A380-841 G-XLEB British Airways (10424102995).jpg', 'UKpop.svg', 'Anglospeak.svg', "Royal Aberdeen Children's Hospital.jpg", 'CHANDOS3.jpg',

### 25. テンプレートの抽出

> 記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

基礎情報は以下のようになっている。

```
{{基礎情報 国
|略名  =イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
|国旗画像 = Flag of the United Kingdom.svg
|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]

<<中略>>

}}
```

`|`を頼りに検索するため，先読みアサーション系統を用いた([pythonのドキュメント](https://docs.python.org/ja/3/library/re.html)より以下引用)

> (?=...)
... が次に続くものにマッチすればマッチしますが、文字列をまったく消費しません。これは 先読みアサーション (lookahead assertion) と呼ばれます。例えば、Isaac (?=Asimov) は 'Isaac ' に、その後に 'Asimov' が続く場合にのみ、マッチします。

ここに関しては，[素人の言語処理](https://qiita.com/segavvy/items/e402ad0a5b0f52453d7f)を参考にさせていただいた。
全く同じになってもつまらないので，{{基礎情報*}}の部分を抽出しないで書いてみた。ただし，他の例においての応用生が低いので，きちんと抽出してからやったほうがいいと思う。具体的には，直接抽出した際に邪魔になるのが`|style=*`という部分だけだったので，それを除去する形で抽出した。

In [202]:
def extract_basic_info(wiki_text):
    result = {}
    p = re.compile(r'^\|(?!style)(\w+?)\s*\=\s*(.+?)(?:(?=\n\|))', re.MULTILINE)
    basics = p.findall(wiki_text)
    for item in basics:
        result[item[0]] = item[1]
    return result

basic_info = extract_basic_info(uk_article)
print(json.dumps(basic_info, sort_keys=True, indent=4, ensure_ascii=False)[0:400])

{
    "GDP値": "2兆3162億<ref name=\"imf-statistics-gdp\" />",
    "GDP値MER": "2兆4337億<ref name=\"imf-statistics-gdp\" />",
    "GDP値元": "1兆5478億<ref name=\"imf-statistics-gdp\">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a=IMF>Data and Statistics>World Economic Ou


### 26. 強調マークアップの除去

> 25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）

''強調''のように，二つ以上の'に囲まれているところが強調マークアップである。処理時ということなので新しく重複する処理を行う関数を定義した。

In [203]:
def extract_basic_removed_reinforce(wiki_text):
    result = {}
    ps = re.compile(r'\'{2,}') # 加えた部分
    p = re.compile(r'^\|(?!style)(\w+?)\s*\=\s*(.+?)(?:(?=\n\|))', re.MULTILINE)
    removed_text = ps.sub('', wiki_text) # 加えた部分
    basics = p.findall(removed_text)
    for item in basics:
        result[item[0]] = item[1]
    return result

basic_info = extract_basic_removed_reinforce(uk_article)
print(json.dumps(basic_info, sort_keys=True, indent=4, ensure_ascii=False)[0:400])

{
    "GDP値": "2兆3162億<ref name=\"imf-statistics-gdp\" />",
    "GDP値MER": "2兆4337億<ref name=\"imf-statistics-gdp\" />",
    "GDP値元": "1兆5478億<ref name=\"imf-statistics-gdp\">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a=IMF>Data and Statistics>World Economic Ou


### 27. 内部リンクの除去

> 26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ

リンクは

```
[[記事名]]
[[記事名|表示文字]]
[[記事名#節名|表示文字]]
```

の３種類がある。処理する際に巻き込みそうな他のwikiマークアップとしては，カテゴリやファイル指定，リダイレクト要素がある。記事にredirectが見当たらなかったので，categoryとファイル指定の除去だけ考える。
ここまで，全て一つの関数で行っているが，本来なら基礎助用法の読み取りなど，一つ一つを別の関数に定義するべきである，と思う。

```
[[Category:ヘルプ|はやみひよう]]
[[ファイル:Wikipedia-logo-v2-ja.png|thumb|説明文]]
#REDIRECT [[記事名]]
#REDIRECT [[記事名#節名]]
```

`?!`：当てはまらない時に正
`?:`：キャプチャしないところを指定

In [204]:
# リンク除去の関数
def remove_links(text):
    p = re.compile(r'\[\[(?!Category\:ファイル)(?:[^|]*?\|)?([^|]*?)\]\]')
    return p.sub(r'\1', text)

def extract_basic_not_link_reinforce(wiki_text):
    result = {}
    ps = re.compile(r'\'{2,}') 
    p = re.compile(r'^\|(?!style)(\w+?)\s*\=\s*(.+?)(?:(?=\n\|))', re.MULTILINE) 
    removed_text = remove_links(ps.sub('', wiki_text)) #　変更した部分
    basics = p.findall(removed_text)
    for item in basics:
        result[item[0]] = item[1]
    return result

basic_info = extract_basic_not_link_reinforce(uk_article)
print(json.dumps(basic_info, sort_keys=True, indent=4, ensure_ascii=False)[0:400])

{
    "GDP値": "2兆3162億<ref name=\"imf-statistics-gdp\" />",
    "GDP値MER": "2兆4337億<ref name=\"imf-statistics-gdp\" />",
    "GDP値元": "1兆5478億<ref name=\"imf-statistics-gdp\">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a=IMF>Data and Statistics>World Economic Ou


### 28. MediaWikiマークアップの除去

> 27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

残っている除去項目は，
27であえて残したファイルと内部リンクの除去，
langタグの除去，
<br/>タグ，<ref>タグの除去
    
基本情報の抜き出しから改めて定義して行ってみる。cite webやcenterがまだ取り出せていないので修正する必要がある。

In [205]:
# 基礎情報の抽出
def extract_basic(text):
    p = re.compile(r'^\|(?!style)(\w+?)\s*\=\s*(.+?)(?:(?=\n\|))', re.MULTILINE)
    basics = p.findall(text)
    return basics

# 除去関数
def remove_emphasis(text):
    p = re.compile(r'\'{2,}')
    return p.sub(r'', text)
def remove_links(text):
    p = re.compile(r'\[\[(?:[^|]*?\|)*?([^|]*?)\]\]')
    return p.sub(r'\1', text)
def remove_tags(text):
    p = re.compile(r'<[^>]*?>')
    return p.sub(r'', text)
def remove_lang(text):
    p = re.compile(r'\{\{lang(?:[^|]*?\|)*?([^|]*?)\}\}')
    return p.sub(r'\1', text)
def remove_ex_link(text):
    p = re.compile(r'\[http:\/\/(?:[^\s]*?)\s([^]]*?)\]')
    return p.sub(r'\1', text)


def main():
    basic_dict = {}
    basic_list = extract_basic(uk_article)
    for target in basic_list:
        explanation = remove_emphasis(target[1])
        explanation = remove_links(explanation)
        explanation = remove_tags(explanation)
        explanation = remove_lang(explanation)
        explanation = remove_ex_link(explanation)
        basic_dict[target[0]] = explanation
    print(json.dumps(basic_dict, sort_keys=True, indent=4, ensure_ascii=False)[0:400])
        
if __name__ == '__main__':
    main()

{
    "GDP値": "2兆3162億",
    "GDP値MER": "2兆4337億",
    "GDP値元": "1兆5478億and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom",
    "GDP統計年": "2012",
    "GDP統計年MER": "2012",
    "GDP統計年元": "2012",
    "GDP順位": "6",
    "GDP順位MER": "6",
    "ccTLD": ".uk / .gb使用は.ukに比べ圧倒的少数。",
    "人口値": "6643万5600{{Cite web|url=https://www.ons.gov.uk/peoplepopulationandcommunity/populationan


### 29. 国旗画像のURLを取得する

> テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki API](http://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Imageinfo)を呼び出して，ファイル参照をURLに変換すればよい）

28の時点で`"国旗画像": "Flag of the United Kingdom.svg"`が取得できている。

In [199]:
import requests

def extract_basic_dict(article):
    basic_dict = {}
    basic_list = extract_basic(article)
    for target in basic_list:
        explanation = remove_emphasis(target[1])
        explanation = remove_links(explanation)
        explanation = remove_tags(explanation)
        explanation = remove_lang(explanation)
        explanation = remove_ex_link(explanation)
        basic_dict[target[0]] = explanation
    return basic_dict

basic_dict = extract_basic_dict(uk_article)
fname_flag = basic_dict['国旗画像']

def obtain_url(basic_dict, title):
    fname_flag = basic_dict[title].replace(' ', '_')
    url = 'https://en.wikipedia.org/w/api.php?' \
        + 'action=query' \
        + '&titles=File:' + fname_flag \
        + '&prop=imageinfo' \
        + '&iiprop=url' \
        + '&format=json'
    data = requests.get(url)
    return re.search(r'"url":"(.+?)"', data.text).group(1)


def main():
    basic_dict = extract_basic_dict(uk_article)
    query_url = obtain_url(basic_dict, "国旗画像")
    print(query_url)
    
if __name__ == '__main__':
    main()

https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg
